In [3]:
#import libraries
import teradata as td
import pandas as pd
import numpy as np
from getpass import getpass, getuser

In [4]:
#log in to Teradata
udaExec = td.UdaExec(appName="Pfizer_KOL_CMS_App", version="1.0",
        logConsole=False)

uname = getuser()
print(uname)
pword = getpass("Teradata Password: ")
 
session = udaExec.connect(method="odbc", system="amrndhp3",
        username=uname, password=pword);

WEITEM
Teradata Password: ········


In [5]:
# get CMS data - takes about 30 seconds
listed_data = []
data = session.execute("SELECT * FROM HVAD_TEAM_T.mw_cms_kol_all_2k17")
for row in data:
    listed_data.append(row[0:])

In [6]:
# get column names
col_names = []
col_names_ter = session.execute("SELECT ColumnName FROM dbc.ColumnsV WHERE DatabaseName='HVAD_TEAM_T' AND TableName='mw_cms_kol_all_2k17' ORDER BY ColumnId")
for item in col_names_ter:
    col_names.append(item[0])
col_names

['record_id',
 'phys_full_name',
 'assoc_type',
 'product',
 'payment_date',
 'amount',
 'num_payments',
 'payment_nature',
 'Payment_Year',
 'submitting_company',
 'paying_company',
 'consolidating_company']

In [7]:
# make dataframe
df = pd.DataFrame(data=listed_data, columns = col_names)
df.head()

,record_id,phys_full_name,assoc_type,product,payment_date,amount,num_payments,payment_nature,Payment_Year,submitting_company,paying_company,consolidating_company
0,389957222,"Soloman, Nehad",recipient,PROLIA,2016-10-06,123.19,1,Food and Beverage,2016,Amgen Inc.,Amgen Inc.,Amgen Inc.
1,31093345,"Strand, Vibeke",recipient,None,2013-11-06,65.07,1,Travel and Lodging,2013,"AbbVie, Inc.","AbbVie, Inc.","AbbVie, Inc."
2,309188896,"Aelion, Jacob",recipient,HUMIRA,2015-02-25,12.95,1,Food and Beverage,2015,"AbbVie, Inc.","AbbVie, Inc.","AbbVie, Inc."
3,359329330,"Lambert, Rhonda E",recipient,KRYSTEXXA,2016-08-12,43.14,1,Food and Beverage,2016,Horizon Pharma plc,Horizon Pharma plc,Horizon Pharma plc
4,388751724,"Huffstutter, Joseph E",recipient,ENBREL,2016-11-11,12.40,1,Food and Beverage,2016,Amgen Inc.,Amgen Inc.,Amgen Inc.


In [11]:
# # clean dataframe
df.dropna()
# df = df.drop(['submitting_company', 'paying_company'], axis=1)

# for colname in ['phys_full_name','assoc_type','payment_date','product','payment_nature','consolidating_company']:
#     dkeys = df[colname].unique()
#     dvalues = list(range(len(df[colname].unique())))
#     dmapping = dict(zip(dkeys, dvalues))
#     df = df.replace({colname: dmapping})

# df.head()

,record_id,phys_full_name,assoc_type,product,payment_date,amount,num_payments,payment_nature,Payment_Year,submitting_company,paying_company,consolidating_company
0,389957222,"Soloman, Nehad",recipient,PROLIA,2016-10-06,123.19,1,Food and Beverage,2016,Amgen Inc.,Amgen Inc.,Amgen Inc.
2,309188896,"Aelion, Jacob",recipient,HUMIRA,2015-02-25,12.95,1,Food and Beverage,2015,"AbbVie, Inc.","AbbVie, Inc.","AbbVie, Inc."
3,359329330,"Lambert, Rhonda E",recipient,KRYSTEXXA,2016-08-12,43.14,1,Food and Beverage,2016,Horizon Pharma plc,Horizon Pharma plc,Horizon Pharma plc
4,388751724,"Huffstutter, Joseph E",recipient,ENBREL,2016-11-11,12.40,1,Food and Beverage,2016,Amgen Inc.,Amgen Inc.,Amgen Inc.
5,398535362,"Salmon, Jane E",recipient,ORENCIA,2016-12-07,4488.00,1,Consulting Fee,2016,Bristol-Myers Squibb Company,"E.R. Squibb & Sons, L.L.C.",Bristol-Myers Squibb Company
6,318144418,"Wright, Grace C",recipient,RITUXAN,2015-05-26,11.55,1,Food and Beverage,2015,"Genentech, Inc.","Genentech USA, Inc.",Roche / Genentech
8,322485644,"Markenson, Joseph",recipient,XELJANZ,2015-08-25,70.42,1,Food and Beverage,2015,Pfizer Inc.,Pfizer Inc.,Pfizer Inc.
9,270224420,"Abubaker, Mohammed",recipient,SIMPONI,2015-05-06,11.74,1,Food and Beverage,2015,"Janssen Pharmaceuticals, Inc","Janssen Biotech, Inc.",Johnson & Johnson and Subsidiaries
11,142531712,"Hamburger, Max",recipient,KRYSTEXXA,2014-05-16,1500.00,1,Compensation for services other than consultin...,2014,Crealta Pharmaceuticals LLC,Crealta Pharmaceuticals LLC,Crealta Pharmaceuticals LLC
13,207312004,"Wells, Alvin",recipient,XELJANZ,2014-12-04,266.67,1,Compensation for services other than consultin...,2014,Pfizer Inc.,Pfizer Inc.,Pfizer Inc.


In [9]:
# from sklearn.model_selection import train_test_split
# from sklearn.ensemble import IsolationForest

# X_train, X_test = train_test_split(df[['record_id','phys_full_name','assoc_type','product','payment_date','amount','num_payments',
#                                       'payment_nature','Payment_Year','consolidating_company']])
# Forest = IsolationForest()
# Forest.fit(X_train)
# y_pred_train = Forest.predict(X_train)
# y_pred_test = Forest.predict(X_test)

#df[(np.abs(df-df.mean())>(3*df.std()))] #keep only the outliers (outside +3 to -3 standard deviations)


In [ ]:
categorical_data = df[['phys_full_name','assoc_type','payment_date','product','payment_nature','submitting_company',
                       'paying_company','consolidating_company']]
numerical_data = df.drop(['phys_full_name','assoc_type','payment_date','product','payment_nature',
                          'consolidating_company','submitting_company','paying_company'], axis=1)

print("testing")
# numerical_data.head()[(np.abs(df-df.mean())>(3*df.std()))] #keep only the outliers (outside +3 to -3 standard deviations)
df.mean()